In [3]:
import json

In [56]:
with open('../hallucinations_filtered.json', 'r') as file:
    # Read the content of the file
    hallucinations_filtered = json.load(file)
    
print(len(hallucinations_filtered))

2753


In [57]:
invented = "<span style='background-color: #BFE9B9; border-bottom: 1px dotted;'>invented</span>"
subjective = "<span style='background-color: #D3D3D3; border-bottom: 1px dotted;'>subjective</span>"
entity = "<span style='background-color: #E9A2D9; border-bottom: 1px dotted;'>entity</span>"
contradictory = "<span style='background-color: #FFFF9B; border-bottom: 1px dotted;'>contradictory</span>"
relation = "<span style='background-color: #F3B78B; border-bottom: 1px dotted;'>relation</span>"
unverifiable = "<span style='background-color: #D3D3D3; border-bottom: 1px dotted;'>unverifiable</span>"

pattern_types = [invented, subjective, entity, contradictory, relation, unverifiable]

In [70]:
entity_hallucinations = []
invented_hallucinations = []
subjective_hallucinations = []
entity_invented_subjective_hallucinations = []
other_hallucinations = []
no_hallucinations = []

# Function to check and classify hallucinations by specific pattern conditions
for hallucination in hallucinations_filtered:
    prediction_result = hallucination["prediction_result"]
    has_invented = invented in prediction_result
    has_subjective = subjective in prediction_result
    has_entity = entity in prediction_result
    # Identifying any other patterns that are not invented, subjective, or entity
    has_other_hallucinations = any(pattern in prediction_result for pattern in pattern_types if pattern not in [invented, subjective, entity])

    # Resetting no_hallucinations for each hallucination check
    is_no_hallucination = all(pattern not in prediction_result for pattern in pattern_types)

    if has_entity and not has_subjective and not has_invented and not has_other_hallucinations:
        entity_hallucinations.append(hallucination)
    elif has_invented and not has_entity and not has_subjective and not has_other_hallucinations:
        invented_hallucinations.append(hallucination)
    elif has_subjective and not has_entity and not has_invented and not has_other_hallucinations:
        subjective_hallucinations.append(hallucination)
    elif (has_invented or has_subjective or has_entity) and not has_other_hallucinations:
        entity_invented_subjective_hallucinations.append(hallucination)
    elif has_other_hallucinations:
        other_hallucinations.append(hallucination)
    elif is_no_hallucination:
        no_hallucinations.append(hallucination)
            
print(len(entity_hallucinations))
print(len(invented_hallucinations))
print(len(subjective_hallucinations))
print(len(entity_invented_subjective_hallucinations))
print(len(other_hallucinations))
print(len(no_hallucinations))

318
121
76
57
650
1531


In [ ]:
def write_results_to_file(results, output_file):
    with open(output_file, "w") as json_file:
         json.dump(results, json_file)
    print(f"Results have been saved to {output_file}")

# entity_hallucinations comes under "factual inaccuracy" pattern and "misclassification" pattern    
write_results_to_file(entity_hallucinations, "factual_inaccuracy_and_misclassification_hallucinations.json")

# invented_hallucinations comes under "fabricated detail" pattern    
write_results_to_file(invented_hallucinations, "fabricated_detail_hallucinations.json")

# subjective_hallucinations comes under "speculative reasoning" pattern    
write_results_to_file(subjective_hallucinations, "speculative_reasoning_hallucinations.json")

# entity_invented_subjective_hallucinations comes under "factual inaccuracy", "misclassification", "fabricated detail", and "speculative reasoning" patterns    
write_results_to_file(entity_invented_subjective_hallucinations, "all_four_patterns_hallucinations.json")

# other_hallucinations comes under "miscellaneous"
# From these we need to find more patterns in future   
write_results_to_file(other_hallucinations, "miscellaneous_hallucinations.json")

# no_hallucinations means there are no hallucinations among these responses
# Needs manual evaluation on this in future  
write_results_to_file(no_hallucinations, "no_hallucinations.json")